In [ ]:
# Mounting drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Path to directory
import os
os.chdir('/content/drive/My Drive/Analytics India/Collaborative Filtering')

**Item Based Filtering**

In [ ]:
import pandas as pd
movies = pd.read_csv("movies.csv",encoding="Latin1")
Ratings = pd.read_csv("ratings.csv")
Tags = pd.read_csv("tags.csv",encoding="Latin1")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings = pd.merge(movies,Ratings).drop(['genres','timestamp'],axis=1)
print(ratings.shape)
ratings.head()

(264505, 4)


,movieId,title,userId,rating
0,1,Toy Story (1995),12882,4.0
1,1,Toy Story (1995),121987,4.5
2,1,Toy Story (1995),18127,4.0
3,1,Toy Story (1995),17318,4.0
4,1,Toy Story (1995),69519,4.0


In [ ]:
UserRatings = ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
UserRatings.head()
print("Before: ",UserRatings.shape)
UserRatings = UserRatings.dropna(thresh=10, axis=1).fillna(0,axis=1)
#userRatings.fillna(0, inplace=True)
print("After: ",UserRatings.shape)

Before:  (862, 2500)
After:  (862, 2487)


**Pearson Correlation**

In [ ]:
corrMatrix = UserRatings.corr(method='pearson')
corrMatrix.head(10)

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),127 Hours (2010),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),15 Minutes (2001),16 Blocks (2006),1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)","20,000 Leagues Under the Sea (1954)",200 Cigarettes (1999),2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),21 (2008),21 Grams (2003),25th Hour (2002),27 Dresses (2008),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),300 (2007),3000 Miles to Graceland (2001),"39 Steps, The (1935)",3:10 to Yuma (2007),40 Days and 40 Nights (2002),"40-Year-Old Virgin, The (2005)","400 Blows, The (Les quatre cents coups) (1959)",48 Hrs. (1982),50 First Dates (2004),50/50 (2011),...,Withnail & I (1987),Witness (1985),Witness for the Prosecution (1957),"Wizard of Oz, The (1939)",Wolf (1994),"Wolf of Wall Street, The (2013)",Women on the Verge of a Nervous Breakdown (Mujeres al borde de un ataque de nervios) (1988),Wonder Boys (2000),Working Girl (1988),"World According to Garp, The (1982)","World Is Not Enough, The (1999)",Wreck-It Ralph (2012),"Wrestler, The (2008)",Wyatt Earp (1994),"X-Files: Fight the Future, The (1998)",X-Men (2000),X-Men Origins: Wolverine (2009),X-Men: Days of Future Past (2014),X-Men: First Class (2011),X-Men: The Last Stand (2006),X2: X-Men United (2003),"Year of Living Dangerously, The (1982)",Yellow Submarine (1968),Yes Man (2008),Yojimbo (1961),You Can Count on Me (2000),You Only Live Twice (1967),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Zack and Miri Make a Porno (2008),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",1.000000,0.079852,0.393357,0.177437,0.044855,0.121371,0.192810,0.093634,0.072082,0.054784,0.157268,0.137365,0.156919,0.166879,0.165558,0.143843,0.142656,0.086394,0.082401,0.227986,0.074479,0.071817,0.044804,0.043106,0.074987,0.060593,0.108225,0.197163,0.109168,0.071595,0.092918,0.150146,0.127611,0.117302,0.119182,0.118063,0.141000,0.179941,0.115751,0.067626,...,0.118765,0.143437,-0.003397,0.162177,0.113693,0.001051,0.043881,0.020002,0.187868,0.055486,0.136661,0.027892,0.040157,0.144538,0.076618,0.083942,0.069078,0.019853,0.023649,0.122995,0.111481,-0.011100,0.101252,0.104034,0.047245,0.046289,0.093793,0.091381,0.165022,0.194754,0.262244,0.075142,0.071867,0.027865,0.103637,0.079652,0.139854,0.088440,0.108367,0.331894
(500) Days of Summer (2009),0.079852,1.000000,-0.027604,0.230017,0.024757,0.097780,0.064554,0.097547,0.120659,0.342501,0.202566,0.022548,0.169530,0.014963,0.116932,0.011658,0.100033,0.056324,0.018133,0.042429,0.064372,-0.044910,0.154622,0.250149,0.145614,0.258035,0.307399,0.036866,0.216285,0.161543,0.162824,0.075686,0.079698,0.174624,0.111525,0.215324,0.084495,0.027922,0.246323,0.528613,...,0.100258,0.034460,0.051940,0.093110,0.024800,0.259812,0.044797,0.185390,0.104904,0.062480,0.090101,0.216299,0.353371,0.051675,0.024107,0.078016,0.109423,0.191781,0.185425,0.141722,0.102176,0.035877,-0.002923,0.296637,-0.008523,0.174548,0.053479,0.142118,0.042587,0.026892,0.021482,0.069279,0.230038,0.089167,0.245939,0.356606,0.238229,0.105943,0.093795,0.091356
*batteries not included (1987),0.393357,-0.027604,1.000000,0.197842,0.083206,0.129024,0.294452,0.127583,0.034409,0.003070,0.127042,0.139577,0.080568,0.167112,0.120290,0.144712,0.165259,0.063558,0.187011,0.145417,0.081506,0.057212,0.063480,0.055050,-0.011400,0.046892,0.116367,0.183776,0.095434,0.076550,0.124170,0.172638,0.136344,0.052077,0.092215,0.108924,0.029417,0.122902,0.070912,0.02

In [ ]:
def get_similar(movie_name,rating):
    similar_ratings = corrMatrix[movie_name]*(rating-2.5)
    similar_ratings = similar_ratings.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_ratings

In [ ]:
#Similar Romantic Movies
romantic_movies= [("Reader, The (2008)",5),("Alice in Wonderland (2010)",3),("Aliens (1986)",1),("2001: A Space Odyssey (1968)",2)]
similar_movies = pd.DataFrame()
for movie,rating in romantic_movies:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

similar_movies.head(10)

,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),127 Hours (2010),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),15 Minutes (2001),16 Blocks (2006),1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)","20,000 Leagues Under the Sea (1954)",200 Cigarettes (1999),2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),21 (2008),21 Grams (2003),25th Hour (2002),27 Dresses (2008),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),300 (2007),3000 Miles to Graceland (2001),"39 Steps, The (1935)",3:10 to Yuma (2007),40 Days and 40 Nights (2002),"40-Year-Old Virgin, The (2005)","400 Blows, The (Les quatre cents coups) (1959)",48 Hrs. (1982),50 First Dates (2004),50/50 (2011),...,Withnail & I (1987),Witness (1985),Witness for the Prosecution (1957),"Wizard of Oz, The (1939)",Wolf (1994),"Wolf of Wall Street, The (2013)",Women on the Verge of a Nervous Breakdown (Mujeres al borde de un ataque de nervios) (1988),Wonder Boys (2000),Working Girl (1988),"World According to Garp, The (1982)","World Is Not Enough, The (1999)",Wreck-It Ralph (2012),"Wrestler, The (2008)",Wyatt Earp (1994),"X-Files: Fight the Future, The (1998)",X-Men (2000),X-Men Origins: Wolverine (2009),X-Men: Days of Future Past (2014),X-Men: First Class (2011),X-Men: The Last Stand (2006),X2: X-Men United (2003),"Year of Living Dangerously, The (1982)",Yellow Submarine (1968),Yes Man (2008),Yojimbo (1961),You Can Count on Me (2000),You Only Live Twice (1967),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Zack and Miri Make a Porno (2008),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
0,0.140827,0.924010,0.008219,0.323203,0.204352,0.203058,0.137776,0.142215,0.477730,0.815311,0.395790,0.102009,0.227321,0.221342,0.380738,0.081612,0.381540,0.052331,0.167845,0.232929,0.222904,0.115093,0.330368,0.548021,0.535384,0.419535,0.476893,0.197049,0.266464,0.117549,0.298929,0.326815,0.243166,0.644921,0.310556,0.234698,0.167326,0.289298,0.439643,0.629596,...,0.092688,0.279168,0.319312,0.315503,0.239579,0.683556,0.096934,0.494129,0.369445,0.222860,0.207180,0.414214,0.830098,0.330208,0.091914,0.152785,0.352921,0.401416,0.456710,0.223153,0.181325,0.182441,0.237232,0.550322,0.050828,0.552878,0.284608,0.281284,0.213745,0.086884,0.035751,0.169532,0.401134,0.231184,0.592630,0.349840,0.294727,0.257824,0.172661,0.262785
1,0.074966,0.163476,0.044820,0.126778,0.077290,0.041254,0.059282,0.038574,0.066210,0.136241,0.095189,0.048523,0.118323,0.076869,0.090185,0.032391,0.058287,0.083807,0.014781,0.035199,0.022711,0.046002,0.133207,0.100498,0.092764,0.093151,0.112383,0.027615,0.102006,0.105152,0.113492,0.061351,0.016065,0.100364,0.078269,0.083556,0.020625,0.018399,0.132010,0.154553,...,0.025639,0.016904,0.017503,0.079965,0.024349,0.117248,-0.004396,0.047481,0.036911,0.005900,0.059943,0.099012,0.127897,0.043233,0.040478,0.103697,0.156644,0.072560,0.164353,0.121278,0.081677,-0.005617,0.043930,0.142246,0.015522,0.037731,0.041792,0.057330,0.047245,0.021841,0.013931,0.006177,0.106898,0.057929,0.098741,0.204000,0.139456,0.047591,0.097460,0.059061
2,-0.189569,-0.054578,-0.185908,-0.172451,-0.172276,-0.168314,-0.279581,-0.027836,-0.247141,-0.235213,-0.094331,-0.390984,-0.266492,-0.247873,-0.173016,-0.383323,-0.238742,-0.132317,-0.299908,-0.126841,-0.595358,-0.323336,-0.156996,-0.149121,-0.202707,-0.263855,-0.102618,-0.171815,-0.497145,-0.396110,-0.374396,-0.160785,-0.202266,-0.319994,-0.104055,-0.291264,-0.167202,-0.293915,-0.219641,-0.036577,...,-0.210008,-0.412411,-0.088121,-0.402760,-0.290390,-0.112266,-0.121725,-0.308928,-0.206551,-0.246010,-0.376959,-0.141748,-0.

In [ ]:
#Similar movies
similar_movies.sum().sort_values(ascending=False).head(20)

Reader, The (2008)                 2.519538
Alice in Wonderland (2010)         1.030416
Blind Side, The  (2009)            1.026193
Doubt (2008)                       1.011005
Midnight in Paris (2011)           1.002829
Help, The (2011)                   1.001966
(500) Days of Summer (2009)        1.000723
Vicky Cristina Barcelona (2008)    0.964360
Milk (2008)                        0.943045
Social Network, The (2010)         0.918004
Changeling (2008)                  0.901801
Argo (2012)                        0.901166
King's Speech, The (2010)          0.870493
Slumdog Millionaire (2008)         0.853153
Up in the Air (2009)               0.849543
Silver Linings Playbook (2012)     0.846293
Intouchables (2011)                0.844388
Life of Pi (2012)                  0.840448
Notebook, The (2004)               0.835141
Moonrise Kingdom (2012)            0.832102
dtype: float64

In [ ]:
#Similar Action Movies
action_movies = [("Skyfall (2012)",5),("Mission: Impossible III (2006)",4),("Toy Story 3 (2010)",2),("2 Fast 2 Furious (Fast and the Furious 2, The) (2003)",4)]
similar_movies = pd.DataFrame()
for movie,rating in action_movies:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

similar_movies.head(10)
similar_movies.sum().sort_values(ascending=False).head(20)

Skyfall (2012)                                           2.808555
Mission: Impossible III (2006)                           2.223072
2 Fast 2 Furious (Fast and the Furious 2, The) (2003)    1.980224
Quantum of Solace (2008)                                 1.646215
Looper (2012)                                            1.641091
Sherlock Holmes (2009)                                   1.636412
Iron Man 3 (2013)                                        1.625235
Mission: Impossible - Ghost Protocol (2011)              1.600969
Dark Knight Rises, The (2012)                            1.579808
Sherlock Holmes: A Game of Shadows (2011)                1.554755
Captain America: The First Avenger (2011)                1.522129
S.W.A.T. (2003)                                          1.502761
Salt (2010)                                              1.486026
Gone in 60 Seconds (2000)                                1.484542
Avengers, The (2012)                                     1.484004
Django Unc